In [ ]:
%matplotlib notebook

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import math
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

np.random.seed(1)

def splitData(X, rate):
    X_train = X[int(X.shape[0] * rate):]
    X_val = X[:int(X.shape[0] * rate)]
    return X_train, X_val

def shuffle(X):
    np.random.seed(10)
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList]

def normalize(x_train):
    x_train_norm = (x_train - np.mean(x_train, axis=0)) / (np.max(x_train, axis=0) - np.min(x_train, axis=0))
    return x_train_norm

def denormalize(norm_value, mean, range_):
    denorm_value = norm_value * range_ + mean
    return denorm_value

def judge(AL):
    output = np.where((AL <= 0.5) , 0, 1)
    return output

def generate_data(num_samples):
    dataset = []
    x_range1 = [-0.5, 0.2]
    x_range2 = [0.8, 1.5]
    y_range1 = [-0.5, 0.2]
    y_range2 = [0.8, 1.5]

    x_values = np.concatenate((np.random.uniform(x_range1[0], x_range1[1], num_samples//2),
                               np.random.uniform(x_range2[0], x_range2[1], num_samples//2)))

    y_values = np.concatenate((np.random.uniform(y_range1[0], y_range1[1], num_samples//2),
                               np.random.uniform(y_range2[0], y_range2[1], num_samples//2)))

    labels = np.where((x_values >= 0.8) & (y_values >= 0.8), 1, 0)
    dataset = np.column_stack((x_values, y_values, labels))

    return dataset

# 生成
dataset = generate_data(10000)


dataset = shuffle(dataset)

print("訓練數據:")
for i in range(5):
    print("樣本:", dataset[i])
dataset_train, dataset_val = splitData(dataset, 0.2)

train_label = dataset_train[:, -1]
# train_label_norm = normalize(train_label)

x_train = dataset_train[:, 0:2]
x_train_norm = normalize(x_train)

test_label = dataset_val[:, -1]
# test_label_norm = normalize(test_label)

x_val = dataset_val[:, 0:2]
x_val_norm = normalize(x_val)

print(x_train_norm)
print(train_label)
print(x_val_norm)
print(test_label)
x_train=x_train.T
x_train_norm=x_train_norm.T
x_train.shape[0]
x_train_norm.shape[0]
train_label = train_label.reshape(-1, 1)


串流輸出內容已截斷至最後 5000 行。
 [ 0.26214291  0.43874434]
 [-0.29039528 -0.33424297]
 [-0.17165348 -0.35612651]
 [ 0.29693788  0.21971359]
 [ 0.48079246  0.19714892]
 [ 0.39323945  0.43970377]
 [-0.31900883 -0.38703179]
 [ 0.45004894  0.32574162]
 [ 0.40952057  0.23009542]
 [-0.36469733 -0.2765143 ]
 [ 0.30987741  0.30728406]
 [ 0.27041213  0.39476495]
 [-0.25915703 -0.24230533]
 [ 0.39859065  0.25780908]
 [-0.23751747 -0.20345515]
 [ 0.30428993  0.23748814]
 [-0.20114083 -0.21007447]
 [-0.30060893 -0.44990497]
 [ 0.30534622  0.25580015]
 [ 0.3814012   0.32953367]
 [ 0.33927786  0.17248765]
 [ 0.49917693  0.16827834]
 [-0.4067368  -0.43778766]
 [-0.41720635 -0.21098026]
 [-0.2707461  -0.26239944]
 [-0.16926234 -0.28227474]
 [ 0.49531252  0.36649481]
 [-0.38298299 -0.33763448]
 [-0.25830124 -0.45565377]
 [-0.46698763 -0.25391483]
 [-0.47686947 -0.22724463]
 [ 0.39711548  0.18840149]
 [ 0.398544    0.25512277]
 [-0.31309762 -0.49969795]
 [-0.38891053 -0.36718424]
 [ 0.44198973  0.20577295]
 [-0.18

In [ ]:
# x_train.shape
train_label.shape

(8000, 1)

In [ ]:
def calculate_average(lst):
    if not lst:
        return None

    total = sum(lst)
    average = total / len(lst)
    return average

In [ ]:
def initialize_parameters(layer_dims):
    np.random.seed(1) #lock random parameters
    parameters = {}
    layers = len(layer_dims)
    for layer in range(1,layers):
        parameters["W"+str(layer)] = np.random.randn(layer_dims[layer], layer_dims[layer-1])
        parameters["b"+str(layer)] = np.zeros((layer_dims[layer], 1))

        assert(parameters['W' + str(layer)].shape == (layer_dims[layer], layer_dims[layer-1]))
        assert(parameters['b' + str(layer)].shape == (layer_dims[layer], 1))

    return parameters



In [ ]:
def linear_forward(A, W, b):
    Z = np.dot(W,A)+b
    cache = (A,W,b)
    return Z, cache

def tanh(Z):
    A = np.tanh(Z)
    cache = Z
    return A, cache

def linear(Z):
    cache = Z
    return Z, cache

In [ ]:
def linear_activation_forward(A_pre, W, b, activation):
    Z, linear_cache = linear_forward(A_pre, W, b)
    if activation == "tanh":
        A = np.tanh(Z)
    elif activation == "linear":
        A = Z
    cache = (linear_cache, A)
    return A, cache

In [ ]:
def model_forward(X, parameters):
    caches = []
    A = X

    # Number of layers
    layers = len(parameters) // 2

    for layer in range(1, layers):
        A_pre = A
        A, cache = linear_activation_forward(A_pre, parameters["W" + str(layer)], parameters["b" + str(layer)], "tanh")
        caches.append(cache)

    AL, cache = linear_activation_forward(A, parameters["W" + str(layers)], parameters["b" + str(layers)], "linear")
    caches.append(cache)

    return AL, caches



In [ ]:
def compute_loss(AL, Y):
    loss = (AL - Y) ** 2 / 2
    loss = np.squeeze(loss)
    return loss


In [ ]:
def linear_backward(dZ, cache):
    A_pre, W, b = cache
    dW = np.dot(dZ, A_pre.T)
    db = dZ
    dA_pre = np.dot(W.T, dZ)
    return dA_pre, dW, db

def tanh_backward(dA, Z):
    dZ = dA * (1 - np.tanh(Z)**2)
    assert dZ.shape == Z.shape
    return dZ

def linear_activation_backward(dA, cache, activation):
    dA_pre = None
    dW = None
    db = None

    linear_cache, activation_cache = cache
    if activation == "tanh":
        dZ = tanh_backward(dA, activation_cache)
        dA_pre, dW, db = linear_backward(dZ, linear_cache)
    elif activation == "linear":
        dA_pre, dW, db = linear_backward(dA, linear_cache)

    return dA_pre, dW, db


In [ ]:
def model_backward(AL, Y, caches, parameters):
    grads = {}
    layers = len(caches)

    # Compute dAL (gradient of the loss function with respect to AL)
    dAL = AL - Y

    # Get the cache of the last layer
    current_cache = caches[layers-1]

    # Backward pass for the last layer
    grads["dA"+str(layers-1)], grads["dW"+str(layers)], grads["db"+str(layers)] = \
               linear_activation_backward(dAL, current_cache, "linear")

    # Backward pass for the hidden layers
    for layer in reversed(range(layers-1)):
        current_cache = caches[layer]
        activation = "tanh" if layer > 0 else "linear"  # Activation function for hidden layers
        grads["dA"+str(layer)], grads["dW"+str(layer+1)], grads["db"+str(layer+1)] = \
             linear_activation_backward(grads["dA"+str(layer+1)], current_cache, activation)

    return grads


In [ ]:
def update_parameters(parameters, grads, learning_rate):
    layers = len(parameters) // 2
    for layer in range(layers):
        dW = grads["dW" + str(layer + 1)]
        db = grads["db" + str(layer + 1)]
        if dW is not None:  # 梯度不為 None 時才更新參數
            parameters["W" + str(layer + 1)] = parameters["W" + str(layer + 1)] - learning_rate * dW
        if db is not None:
            parameters["b" + str(layer + 1)] = parameters["b" + str(layer + 1)] - learning_rate * db
    return parameters


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

plt.rcParams['animation.embed_limit'] = 50  # 設置更大的值（以MB為單位）
def deep_learning_model(X, Y, layer_dims, learning_rate, num_iterations, print_cost=False):
    losses = []
    outputs = []
    mean_loss = []
    parameters = initialize_parameters(layer_dims)
    for i in range(num_iterations):
        for j in range(X.shape[1]):
            x_single = X[:,j:j+1]
            y_single = Y[j]
            AL, caches = model_forward(x_single, parameters)
            loss = compute_loss(AL, y_single)
            grads = model_backward(AL, y_single, caches, parameters)
            parameters = update_parameters(parameters, grads, learning_rate)
            losses.append(loss)
            if(j<1000):
                outputs.append(AL)
        mean_loss.append(calculate_average(losses))
    print(mean_loss)
    return parameters, outputs, mean_loss

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def draw(x_train_norm, outputs, mean_loss, num_iterations, num_input, print_cost=False):
    outputs = np.array(outputs)
    # train_label = train_label.astype(float)
    # 定義錯誤區間
    bins = np.linspace(-0.05, 0.08, num=10)
    hist, bins = np.histogram(mean_loss, bins)

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 4))
    fig2 = plt.figure()

    ax1.set_xlim(0, num_iterations)
    ax1.set_ylim(0, 1)  # 設置初始y軸範圍
    line1, = ax1.plot([], [], lw=2)
    text1 = ax1.text(0.02, 0.95, '', transform=ax1.transAxes)

    ax2.set_xlim(0, num_input)
    ax2.set_ylim(0, np.max(outputs) * 1.1)  # 設置初始y軸範圍
    line2, = ax2.plot([], [], lw=2)
    text2 = ax2.text(0.02, 0.95, '', transform=ax2.transAxes)

    line3 = ax2.plot(np.arange(num_input), train_label[0:num_input], lw=2)
    text3 = ax2.text(0.02, 0.95, '', transform=ax2.transAxes)
    ax4 = fig2.add_subplot(111, projection='3d')


    def init():
        line1.set_data([], [])
        text1.set_text('')
        line2.set_data([], [])
        text2.set_text('')

        return line1, text1, line2, text2

    def update(i):
        # 更新訓練過程中的損失函數
        cost_text = "Cost after iteration %i: %f" % (i, mean_loss[i])
        text1.set_text(cost_text)
        xdata = np.arange(i + 1)  # 限制 x 范围為 0 到 i+1
        ydata = mean_loss[:i + 1]
        line1.set_data(xdata, ydata)
        ax1.set_xlim(0, num_iterations)
        ax1.set_ylim(0, np.max(mean_loss) * 1.1)

        # 更新訓練過程中的損失函數
        cost_text = "Output after iteration %i " % (i)
        text2.set_text(cost_text)
        xdata = np.arange(num_input)  # 限制 x 范围
        ydata = outputs[num_input * i:num_input * (i + 1)]
        line2.set_data(xdata, ydata)
        ax2.set_xlim(0, num_input)
        ax2.set_ylim(0, np.max(outputs) * 1.1)

        return line1, text1, line2, text2


    def update2(i):
      ax4.clear()
      x_data = x_train_norm[0, :num_input]
      y_data = x_train_norm[1, :num_input]
      z_data = outputs[num_input * i:num_input * (i + 1)]
      ax4.scatter(x_data, y_data, z_data, c=z_data, cmap='viridis', marker='o')
      ax4.set_xlim(np.min(x_train_norm[0, :num_input]), np.max(x_train_norm[0, :num_input]))
      ax4.set_ylim(np.min(x_train_norm[1, :num_input]), np.max(x_train_norm[1, :num_input]))
      ax4.set_zlim(np.min(outputs[num_input * i:num_input * (i + 1)]), np.max(outputs[num_input * i:num_input * (i + 1)]))

      ax4.set_xlabel('Feature 1')
      ax4.set_ylabel('Feature 2')
      ax4.set_zlabel('Output')

      return ax4,



    ani = FuncAnimation(fig, update, frames=num_iterations, init_func=init, blit=True)
    ani2 = FuncAnimation(fig2, update2, frames=num_iterations, blit=True)


    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training Loss')

    ax2.set_xlabel('Output Index')
    ax2.set_ylabel('Output Value')
    ax2.set_title('Output Variation')

    # 顯示錯誤直方圖
    ax3.bar(bins[:-1], hist, width=0.1)
    ax3.set_xlabel('Error Interval')
    ax3.set_ylabel('Amount')
    ax3.set_title('Error Histogram')

    # 將動畫轉換為HTML格式並顯示
    html_anim = ani.to_jshtml()
    html_anim2 = ani2.to_jshtml()
    display(HTML(html_anim))
    display(HTML(html_anim2))
    # Show the plot
    plt.show()


In [ ]:
x_train_norm[:,0:1].shape

(2, 1)

In [ ]:
layer_dims = [x_train.shape[0], 2, 1]
learning_rate = 0.001
num_iterations = 250
parameters, outputs, mean_loss= deep_learning_model(x_train_norm, train_label, layer_dims, learning_rate, num_iterations, print_cost=True)#訓練

[0.026449491565067425, 0.016072571352251926, 0.01259776235598819, 0.010856343271048691, 0.009810414451097288, 0.009112833513826043, 0.008614483195168306, 0.00824070348059511, 0.007949986127484331, 0.007717416967223916, 0.007527138729364308, 0.0073685790550603635, 0.00723441836834153, 0.007119428329450256, 0.007019774804272506, 0.006932582188746846, 0.006855651491714219, 0.0067872723874915514, 0.0067260946196069865, 0.006671037983313689, 0.006621228027414351, 0.0065759492919950134, 0.006534610745044836, 0.006496719859942148, 0.006461862914357205, 0.006429689835569282, 0.006399902413496015, 0.006372245039504183, 0.006346497361330384, 0.006322468407018157, 0.00629999184615804, 0.00627892213964573, 0.006259131389484015, 0.00624050674450217, 0.006222948250809169, 0.006206367060504945, 0.006190683930870667, 0.006175827960529164, 0.006161735520042049, 0.006148349342917455, 0.006135617749640829, 0.006123493982558756, 0.006111935633569284, 0.006100904149854881, 0.006090364405517384, 0.006080284

In [ ]:
draw(x_train_norm, outputs, mean_loss, num_iterations, 1000)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#將數組的完整內容顯示出來
np.set_printoptions(threshold=np.inf)
AL, cache=model_forward(x_val_norm.T, parameters)# 測試集向前傳播
print(AL)
# AL1 = denormalize(AL, np.mean(test_label), np.max(test_label) - np.min(test_label))
# print(AL1)
AL2=judge(AL)
print(AL2)#結果

[[-2.00945864e-02  1.08853051e+00 -4.06002661e-02  1.80779244e-01
   9.80907788e-01 -8.53376914e-02  9.12593067e-01 -1.43720552e-01
   3.21609353e-02  1.01889429e+00  1.81715669e-01  1.10154776e-01
   9.20212473e-01  9.39194271e-01  9.88561206e-01  1.32379809e-01
   9.49159031e-02  1.17992356e+00 -5.15389646e-02  9.75794952e-03
   9.71089606e-01  9.17189273e-02 -6.51689946e-02  1.12426526e+00
   7.52846564e-02 -8.37465774e-02  1.14407975e+00  8.19251200e-01
   1.44917472e-01  8.57544953e-01  1.11461894e+00  6.89362034e-02
   9.95505469e-01 -1.00247697e-01  1.11049455e+00  8.87480734e-01
  -1.14812009e-01  1.54426759e-02  1.14954946e+00  9.58241102e-01
   8.18427297e-01  2.07762496e-01 -9.84551632e-03  1.09486444e+00
   1.89096160e-01  1.11363300e+00  7.11385549e-02  4.25188221e-02
   6.90998294e-02 -1.45573916e-02 -4.70264561e-02  2.49679936e-01
   1.05550810e+00 -1.12261675e-02  1.37706328e-01  2.52539932e-01
   8.25774248e-01 -1.39235951e-01  9.72468950e-01  8.89996716e-01
  -1.17612